In [7]:
from selenium import webdriver
import time
import pytube
import os
import pysrt
import re
import deepl_api

with open('deepl_api.txt', 'r') as f:
    api_key = f.read()

In [2]:
channel_url = 'https://www.youtube.com/c/BrightInsight/videos'

driver = webdriver.Chrome('../chromedriver/chromedriver.exe')
driver.get(channel_url)

time.sleep(5)

button = driver.find_element_by_xpath("//button[@aria-label='Agree to the use of cookies and other data for the purposes described']")
button.click()

for i in range(5):
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 100000)")

links = driver.find_elements_by_xpath("//a[@class='yt-simple-endpoint inline-block style-scope ytd-thumbnail'][@rel='null']")
urls = []

for link in links:
    urls.append(link.get_attribute('href'))

time.sleep(2)

driver.close()

In [23]:
def download_video(video_path):
    video.download(video_path)

def download_srt(video, video_path):
    srt_captions = caption.generate_srt_captions()
    srt_name = '{}/{}.srt'.format(video_path, video.default_filename)

    with open(srt_name, "w") as f:
        f.write(srt_captions)

def save_url(video_path, url):
    download_url = '{}/url.txt'.format(video_path)
    with open(download_url, "w") as f:
        f.write(url)

urls = ['https://www.youtube.com/watch?v=LHDSmKvs9Tc']

for url in urls:
    time.sleep(2)
    yt = pytube.YouTube(url)
    video = yt.streams.get_highest_resolution()
    capt = yt.captions
    if capt.lang_code_index.get('en'):
        caption = yt.captions.get_by_language_code('en')
    elif capt.lang_code_index.get('a.en'):
        caption = yt.captions.get_by_language_code('a.en')
    else:
        continue

    vid_title = video.title\
        .replace('\\', ' ').replace('/', ' ').replace(':', ' ')\
        .replace('*', ' ').replace('?', ' ').replace('"', ' ')\
        .replace('<', ' ').replace('>', ' ').replace('|', ' ')\
        .replace('.', '').rstrip()
    vid_title = re.sub(' +', ' ', vid_title)
    video_path = './downloads/{}'.format(vid_title)

    if not os.path.isdir(video_path):
        os.makedirs(video_path)

    dir_files = os.listdir(format(video_path))
    if len(dir_files) >= 3:
        print('Skipping video')
        continue

    is_url = False
    is_video = False
    is_caption = False
    for dir_file in dir_files:
        if dir_file.endswith('.txt'):
            is_url = True
        if dir_file.endswith('.srt'):
            is_caption = True
        if dir_file.endswith('.mp4'):
            is_video = True

    if not is_url:
        save_url(video_path, url)
    if not is_caption:
        download_srt(video, video_path)
    if not is_video:
        download_video(video_path)

<ipython-input-23-df86908adb75>:26: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


In [24]:
from_language = 'EN'
to_language = 'SK'
deepl = deepl_api.DeepL(api_key)

for video_dir in os.listdir('./downloads'):
    video_files = os.listdir('./downloads/{}'.format(video_dir))
    if len(video_files) == 4:
        print('Skip translation')
        continue
    srt_name = None
    for vid in video_files:
        if vid.endswith('.srt'):
            srt_name = vid
            break
    if srt_name is None:
        continue
    srt_file = pysrt.open(f'./downloads/{video_dir}/{srt_name}')

    parts = 1
    video_transcript = []
    new_srt_parts = []
    for srt_part in srt_file:
        video_transcript.append(srt_part.text)
        new_srt_parts.append(srt_part)

        if parts % 40 == 0:
            translations = deepl.translate(source_language=from_language, target_language=to_language, texts=video_transcript)
            for t, p in zip(translations, new_srt_parts):
                p.text = t['text']
            video_transcript = []
            new_srt_parts = []

        parts += 1

    translations = deepl.translate(source_language=from_language, target_language=to_language, texts=video_transcript)
    for t, p in zip(translations, new_srt_parts):
        p.text = t['text']

    new_file_name = srt_name.replace('.mp4.srt', '.srt')
    srt_file.save(f'./downloads/{video_dir}/{new_file_name}', encoding='utf-8')


In [63]:
for video_dir in os.listdir('./downloads'):
    video_files = os.listdir('./downloads/{}'.format(video_dir))

    srt_name = None
    for vid in video_files:
        if vid.endswith('.srt') and not vid.endswith('.mp4.srt'):
            srt_name = vid
            break
    if srt_name is None:
        continue
    srt_file = pysrt.open(f'./downloads/{video_dir}/{srt_name}')

    prev_part = None
    for index, srt_part in enumerate(srt_file):
        if prev_part is None:
            prev_part = srt_part
            continue

        if prev_part.end > srt_part.start:
            prev_time = prev_part.end.minutes * prev_part.end.MINUTES_RATIO + prev_part.end.seconds * prev_part.end.SECONDS_RATIO + prev_part.end.milliseconds
            srt_time = srt_part.start.minutes * srt_part.start.MINUTES_RATIO + srt_part.start.seconds * srt_part.start.SECONDS_RATIO + srt_part.start.milliseconds
            new_time = (prev_time + srt_time) // 2
            prev_part.end.minutes = new_time // prev_part.end.MINUTES_RATIO
            prev_part.end.seconds = (new_time - prev_part.end.minutes * prev_part.end.MINUTES_RATIO) // prev_part.end.SECONDS_RATIO
            prev_part.end.milliseconds = (new_time - prev_part.end.minutes * prev_part.end.MINUTES_RATIO - prev_part.end.seconds * prev_part.end.SECONDS_RATIO)
            srt_part.start.minutes = new_time // prev_part.end.MINUTES_RATIO
            srt_part.start.seconds = (new_time - prev_part.end.minutes * prev_part.end.MINUTES_RATIO) // prev_part.end.SECONDS_RATIO
            srt_part.start.milliseconds = (new_time - prev_part.end.minutes * prev_part.end.MINUTES_RATIO - prev_part.end.seconds * prev_part.end.SECONDS_RATIO)
            if srt_part.start.milliseconds < 999:
                srt_part.start.milliseconds += 1
            else:
                prev_part.end.milliseconds -= 1
            print(f'{index}: {prev_part.end.minutes}:{prev_part.end.seconds}:{prev_part.end.milliseconds}')

        prev_part = srt_part

    srt_file.save(f'./downloads/{video_dir}/edited_{srt_name}')

1: 0:3:345
2: 0:5:295
3: 0:7:845
4: 0:11:70
5: 0:14:160
6: 0:16:875
7: 0:19:709
8: 0:22:334
9: 0:24:390
10: 0:26:220
11: 0:28:650
12: 0:31:515
13: 0:34:230
14: 0:37:169
15: 0:39:479
16: 0:42:194
17: 0:45:44
18: 0:47:819
19: 0:50:579
20: 0:52:800
21: 0:55:140
22: 0:58:515
23: 1:1:545
24: 1:3:615
25: 1:6:270
26: 1:8:520
27: 1:10:20
28: 1:12:0
29: 1:14:595
30: 1:17:100
31: 1:19:575
32: 1:21:790
33: 1:24:310
34: 1:26:384
35: 1:28:49
36: 1:30:254
37: 1:32:280
38: 1:33:975
39: 1:35:775
40: 1:38:10
41: 1:40:19
42: 1:41:919
43: 1:44:259
44: 1:46:674
45: 1:49:270
46: 1:51:495
47: 1:53:684
48: 1:56:279
49: 1:58:559
50: 2:0:809
51: 2:2:984
52: 2:4:859
53: 2:7:34
54: 2:9:704
55: 2:11:998
56: 2:13:425
57: 2:14:505
58: 2:15:930
59: 2:17:565
60: 2:19:470
61: 2:21:659
62: 2:24:344
63: 2:26:655
64: 2:28:905
65: 2:31:245
66: 2:33:345
67: 2:35:625
68: 2:37:560
69: 2:39:510
70: 2:41:565
71: 2:44:55
72: 2:46:185
73: 2:47:920
74: 2:50:109
75: 2:52:259
76: 2:54:615
77: 2:56:760
78: 2:58:260
79: 3:0:90
80: 3:

C:\Programming\python_3.8.9\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
